In [5]:
import numpy as np
import pandas as pd
import cvxpy as cp
from scipy.optimize import minimize
from scipy.stats import gmean

rets=pd.read_excel('50return3.xlsx')
rets=np.array(rets)
def sb_two_zones(rets: np.array, threshold1: float=0.5, threshold2: float=0.1, sev: float=200) -> tuple:
    """
    compute Smyth-Broby enhancement to Gerber statistic
    """
    assert 1 > threshold1 > 0
    assert 1 > threshold2 > 0
    
    n, p = rets.shape
    mean_vec = rets.mean(axis=0)
    sd_vec = rets.std(axis=0)
    SD = np.diag(sd_vec)
    std_rets= (rets-mean_vec)/sd_vec
    
    cov_mat = np.zeros((p, p))  
    cor_mat = np.zeros((p, p))  
    for i in range(p):
        for j in range(i + 1):
            discordant_sum = 0;   concordant_sum = 0;   neutral_sum = 0;  
            for k in range(n):  
                denominator = 1 + np.abs((np.abs(std_rets[k, i])-np.abs(std_rets[k, j])))**sev
                s3 = np.sqrt((1+np.abs(std_rets[k,i]))*(1+np.abs(std_rets[k,j])))/denominator
                if ((np.abs(rets[k, i]) > threshold1*sd_vec[i]) or (np.abs(rets[k, j]) > threshold1*sd_vec[j])) \
                    and (np.abs(std_rets[k, i]) < 4) and (np.abs(std_rets[k, j]) < 4):
                    if ((std_rets[k, i] >= threshold2) and (std_rets[k, j] >= threshold2)) or ((std_rets[k, i] <= -threshold2) and (std_rets[k, j] <= -threshold2)):
                        concordant_sum += s3
                    elif ((std_rets[k, i] >= threshold2) and (std_rets[k, j] <= -threshold2)) or ((std_rets[k, i] <= -threshold2) and (std_rets[k, j] >= threshold2)):
                        discordant_sum += s3
                    elif ((abs(std_rets[k, i]) < threshold2)  and (abs(std_rets[k, j]) >= threshold2)) or ((abs(std_rets[k, i]) >= threshold2)  and (abs(std_rets[k, j]) < threshold2)):
                        neutral_sum += s3
                    
            # compute SB co-movement matrix
            cor_mat[i, j] = (concordant_sum - discordant_sum) / (concordant_sum + discordant_sum + neutral_sum +1e-06)
            cor_mat[j, i] = cor_mat[i, j]
            cov_mat[i, j] = cor_mat[i, j] * sd_vec[i] * sd_vec[j]
            cov_mat[j, i] = cov_mat[i, j]
    
    
    return cov_mat, cor_mat
def minimize_volatility(weights, cov_matrix):
    """
    Returns the portfolio volatility given the weights and the covariance matrix
    """
    m_v=portfolio_volatility(weights, cov_matrix)
    return m_v
def optimize_portfolio(cov_matrix):
    num_stocks = rets.shape[1]
    weights = np.random.random(num_stocks)
    weights /= np.sum(weights)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
                   {'type': 'ineq', 'fun': lambda x: portfolio_volatility(x, cov_matrix) - 0.03})
    bounds = tuple((0, 1) for i in range(num_stocks))
    optimal_weights = minimize(minimize_volatility, weights, args=cov_matrix, method='SLSQP', bounds=bounds, constraints=constraints)
    return optimal_weights.x
def portfolio_returns(returns, weights):
    weights = weights.transpose()
    p_r= np.dot(returns,weights)*12
    return p_r
def portfolio_volatility(weights, cov_matrix):
    """
    Returns the portfolio volatility given the weights and the covariance matrix
    """
    p_v=np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return p_v 
for t in range(24,240):
    ret=rets[t-24:t,:]
    cov,cor=sb_two_zones(ret)
    ox=optimize_portfolio(cov)
    pr=portfolio_returns(ret,ox)
    pv=portfolio_volatility(ox, cov)
    prm=pr.mean()*0.9942
    print(prm)

0.04234565588499231
0.08955180160166432
0.08981702494958367
0.09108171754373894
0.11622519633527427
0.11879635826017028
0.13417004931080184
0.21897510678248985
0.2288772693677739
0.2731175683412349
0.2242388593434198
0.20964772106148832
0.2137037312617561
0.2027550577178298
0.19954997064377056
0.22598211481101516
0.2673005210626754
0.25008551513629407
0.27473192677492986
0.26011798347415316
0.23330596411280483
0.203721077291626
0.18496119447619208
0.18737292179894818
0.19276779540884292
0.15328761753700923
0.10253006256217337
0.02543423781184973
0.05027507286529355
0.10391847512544591
0.08459964845939462
0.08819496597905606
0.08047616820846018
0.012996099896166636
0.11139009364949487
0.08718238015822928
0.1323263977248314
0.12551054963114372
0.10427741273538481
0.10712141659297139
0.08946940392544965
0.10930597217979657
0.09875052112054751
0.12448316512634681
0.15132258438905408
0.0987404535250073
0.0959443368547015
0.08368171475028759
0.11136613123222219
0.10776112108773925
0.06726033

C:\Users\User\AppData\Local\Temp\ipykernel_1544\1482625837.py:20: RuntimeWarning: invalid value encountered in true_divide
  std_rets= (rets-mean_vec)/sd_vec


0.15821690836022523
0.09398032832617377
0.09741509030675492
0.1008406541250805
0.09263009739371607
0.10578242104493378
0.12921482113248334
0.11385134037283907
0.14138234347054254
0.10303957528209638
0.08629665127602633
0.07671474473215946
0.10398006676117948
0.1040310896452492
0.08320046632969832
0.03137973431787181
0.0009531709810076419
-0.027280316911449595
-0.009133307689906717
-0.0077569280680485405
0.018072507816143404
-0.01527962420843635
-0.029896948460312805
0.006639518456902254
0.008091774301015944
-0.011713564334318392
0.012282377656404869
-0.008567827316510866
-0.016450890113344024
0.01208184222396341
0.03593293297129227
0.024052359508658024
-0.0010286602090423104
0.008385845861632877
0.025622834440887907
0.07104003300536565
0.042550259919794726
0.05795924613242915
0.09534950149622466
0.10739763207005036
0.14576521125256087
0.18016560161149672
0.15056574107593676
0.17228508758410632
0.17697958244932932
0.17272596437616064
0.18034522805040384
0.1698354969983875
0.161986951297

In [6]:
def optimize_portfolio(cov_matrix):
    num_stocks = rets.shape[1]
    weights = np.random.random(num_stocks)
    weights /= np.sum(weights)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
                   {'type': 'ineq', 'fun': lambda x: portfolio_volatility(x, cov_matrix) - 0.06})
    bounds = tuple((0, 1) for i in range(num_stocks))
    optimal_weights = minimize(minimize_volatility, weights, args=cov_matrix, method='SLSQP', bounds=bounds, constraints=constraints)
    return optimal_weights.x
for t in range(24,240):
    ret=rets[t-24:t,:]
    cov,cor=sb_two_zones(ret)
    ox=optimize_portfolio(cov)
    pr=portfolio_returns(ret,ox)
    pv=portfolio_volatility(ox, cov)
    prm=pr.mean()*0.9942
    print(prm)

0.045342565226878524
0.1261657254124047
0.09118018873817638
0.12550125579751376
0.18392969817910917
0.22321331483016624
0.27144372502205505
0.3901993861400957
0.36521864793927283
0.43647994030869364
0.3298578855481885
0.33020361043984303
0.31009171555549214
0.30346844098599335
0.29900193802294783
0.3356391176201158
0.3563043943000007
0.28106150444518124
0.3019938481918322
0.33372053718816513
0.27791049189874084
0.300965956416848
0.28062056684736264
0.2272751307514529
0.2471166910668572
0.20595567243301002
0.21123826415626068
0.12876211561657977
0.08721523789342155
0.12199421853955784
0.0872742219585971
0.07083373720276571
0.07817791408992544
0.023729012914960507
0.10968650441821144
0.10599021501178535
0.14988993774860065
0.1466447167034854
0.13311108640984418
0.10991278655972724
0.07097749950464613
0.07734733722622202
0.07580504265032091
0.10013523913084164
0.1765820877800629
0.10347586149316453
0.1792799861103181
0.1148895637196878
0.1863758506331683
0.05465521970358297
0.058529935753

C:\Users\User\AppData\Local\Temp\ipykernel_1544\1482625837.py:20: RuntimeWarning: invalid value encountered in true_divide
  std_rets= (rets-mean_vec)/sd_vec


0.047149521361876154
0.06824584592143686
0.01069640951513993
-0.021167475090650298
0.03032724664540983
0.03352430061855394
0.010873400042753214
0.02552364667577969
0.0482493801989776
0.11759625020744191
0.02051815733519429
0.005421129667746959
0.05013609564078879
0.06386603981085293
-0.045980209515433475
-0.011441422676213411
-0.059792267728029015
-0.09055487170562708
-0.1110947832676051
-0.050890738281695304
-0.045209007881995764
-0.04686591508718912
-0.056628959145371485
-0.03060050681911183
-0.05560884787932484
-0.11681731713031199
-0.06316073778248929
-0.09880835770780935
-0.0646736328007078
-0.00669231068611106
0.005936979694876263
0.015197088862644847
-0.019739726375726116
0.02944274112843371
0.05988328655072931
0.058082348625016184
0.14910377297826782
0.10264688098785954
0.07973871536288159
0.08319251145647419
0.12782073601096272
0.19368404045891593
0.15058316468817712
0.14804460966412453
0.17585602649315407
0.17036647128339155
0.17314890961980436
0.16381790502374405
0.103081202

In [7]:
def optimize_portfolio(cov_matrix):
    num_stocks = rets.shape[1]
    weights = np.random.random(num_stocks)
    weights /= np.sum(weights)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
                   {'type': 'ineq', 'fun': lambda x: portfolio_volatility(x, cov_matrix) - 0.09})
    bounds = tuple((0, 1) for i in range(num_stocks))
    optimal_weights = minimize(minimize_volatility, weights, args=cov_matrix, method='SLSQP', bounds=bounds, constraints=constraints)
    return optimal_weights.x
for t in range(24,240):
    ret=rets[t-24:t,:]
    cov,cor=sb_two_zones(ret)
    ox=optimize_portfolio(cov)
    pr=portfolio_returns(ret,ox)
    pv=portfolio_volatility(ox, cov)
    prm=pr.mean()*0.9942
    print(prm)

0.08120673230984
0.17599529401441677
0.14605505073686842
0.1530353590821687
0.26563599988192665
0.26600385521558195
0.3973247759772617
0.5695940883414128
0.5146177573190442
0.5875348687831156
0.4495798236350469
0.4392215798385624
0.41957912605832903
0.47264873877034586
0.38627366948791664
0.450249747507986
0.4906960316854861
0.43935171856965366
0.5234821223371308
0.5102041718344513
0.4153645789753426
0.3423865359763879
0.36378367359981906
0.27908753234788924
0.3508675684724017
0.25338068616298026
0.2877323955816949
0.18294552399171296
0.0876376968998871
0.12922478819911298
0.055608433315772456
0.012122971522909878
0.06640446911932746
-0.008146636308040564
0.17773028395059579
0.1299190416545115
0.2231375021700148
0.1767817549677374
0.23359963088160635
0.184967659284926
-0.013149039282737057
0.20899094526279907
0.162338347019929
0.07797663615467942
0.010059536204427242
0.017506919516542447
0.013145323606959856
0.018852974020339548
0.08033875893539927
0.04200254793626281
-0.02355101494699

C:\Users\User\AppData\Local\Temp\ipykernel_1544\1482625837.py:20: RuntimeWarning: invalid value encountered in true_divide
  std_rets= (rets-mean_vec)/sd_vec


-0.08145846302586089
-0.06081616614443638
-0.07002209746281493
-0.09251042848662246
-0.04851184003526553
-0.037498287277316626
-0.16839795234717864
-0.08807148413438122
-0.10495176130041867
-0.16424479632108102
0.13618341305064716
-0.121253419413997
-0.13669282114502102
-0.09740953023417606
-0.24124875000208826
-0.18898847219890683
0.0869605136368782
0.02309128920096475
0.02013950940063195
-0.0059110160992800675
-0.07564718669880707
-0.11026224809997125
-0.1460203220007719
-0.09022613550035065
-0.07385911799733845
-0.2111029524190078
0.1476404543997867
-0.20457922586405655
-0.10597227509963802
-0.04052856299938874
0.040781011140000774
-0.04868448269917897
0.03351345804798188
-0.06076898369959071
0.08672859105962671
0.09406134893874733
0.03180175402956904
0.11702524646081419
0.04852538631492879
0.08784644111329656
0.36093547619916166
0.09404038380450744
0.2092480174960296
0.13413705305511014
0.10223027053547709
0.08559021861406854
0.09306683459936035
0.21051512331973576
0.03567218909888

In [8]:
def optimize_portfolio(cov_matrix):
    num_stocks = rets.shape[1]
    weights = np.random.random(num_stocks)
    weights /= np.sum(weights)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
                   {'type': 'ineq', 'fun': lambda x: portfolio_volatility(x, cov_matrix) - 0.12})
    bounds = tuple((0, 1) for i in range(num_stocks))
    optimal_weights = minimize(minimize_volatility, weights, args=cov_matrix, method='SLSQP', bounds=bounds, constraints=constraints)
    return optimal_weights.x
for t in range(24,240):
    ret=rets[t-24:t,:]
    cov,cor=sb_two_zones(ret)
    ox=optimize_portfolio(cov)
    pr=portfolio_returns(ret,ox)
    pv=portfolio_volatility(ox, cov)
    prm=pr.mean()*0.9942
    print(prm)

0.06495523989376674
0.14606093555863214
0.24082860496033034
0.21078340772092835
0.3100376278748872
0.3245332501728281
0.5263952181916266
0.6869483140959788
0.6575518697635058
0.7166792409403397
0.5689002216883701
0.5577818623769036
0.588005903255653
0.5881745566304323
0.5160581927520488
0.5853393025112004
0.7127447171570968
0.5806622196109433
0.7796649741196988
0.696459717668911
0.6173917468540916
0.3610835444841052
0.3460438956399555
0.3494054630853818
0.34487160309339193
0.20712302223307438
0.16989252516349657
0.1860901692910965
0.00424305529649811
0.055557714822324196
-0.0426577566130622
-0.10987392187415308
0.02194977922181376
-0.0737745741767985
0.22224400849532316
0.15538864728207621
0.2804222394923915
0.1972747180469373
0.21996751555310043
0.2131699543783861
0.39328930439212095
0.32858493838317954
0.33464171899576756
0.27221068739733323
0.3108848372939508
0.40676042650741323
-0.008721619342484173
0.001423694403734179
0.07404453630179697
0.022668257102720652
-0.040505199298549024

C:\Users\User\AppData\Local\Temp\ipykernel_1544\1482625837.py:20: RuntimeWarning: invalid value encountered in true_divide
  std_rets= (rets-mean_vec)/sd_vec


0.616891931138376
-0.27411933269755734
-0.2560393085971008
0.04829525341685412
-0.2711422007967622
-0.2755405415988462
-0.26594502029747474
-0.28622868869636525
-0.20046700829811034
0.11093730990457379
0.1372955403044523
-0.2588514032975699
-0.22548306869795184
-0.14367184199738933
0.184810346701311
0.13262130900348906
0.09012373290085124
0.02309128920011454
0.02013950940010833
-0.005911016099745239
0.12681518099869254
0.05022499560007975
-0.12101551529913919
0.10050069539991217
-0.08375538479810313
0.008686325399519112
-0.12805494839856846
-0.08320410090028449
-0.10597227509945548
-0.06421967259453179
-0.03301241099924848
-0.04868448270005983
-0.02835060720014999
-0.06076898370019768
-0.07774445159829763
0.061820222670011776
-0.0650902739998306
-0.0016210430989155118
0.0338714823905082
0.023589939980231952
0.4245008608250412
0.1214479729347193
0.33231251565992004
0.13227483030481132
0.09555952140182025
0.0738501702028886
0.5494789162666209
0.5141387650514406
0.014401008698877449
0.072

In [9]:
def optimize_portfolio(cov_matrix):
    num_stocks = rets.shape[1]
    weights = np.random.random(num_stocks)
    weights /= np.sum(weights)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
                   {'type': 'ineq', 'fun': lambda x: portfolio_volatility(x, cov_matrix) - 0.15})
    bounds = tuple((0, 1) for i in range(num_stocks))
    optimal_weights = minimize(minimize_volatility, weights, args=cov_matrix, method='SLSQP', bounds=bounds, constraints=constraints)
    return optimal_weights.x
for t in range(24,240):
    ret=rets[t-24:t,:]
    cov,cor=sb_two_zones(ret)
    ox=optimize_portfolio(cov)
    pr=portfolio_returns(ret,ox)
    pv=portfolio_volatility(ox, cov)
    prm=pr.mean()*0.9942
    print(prm)

0.10190271702909537
0.2209594942344414
0.27696692918015775
0.3061563438829445
0.3483011255521357
0.428697132867554
0.6438063353613961
0.787229400604013
0.8007529586300711
0.8934628475691155
0.7353618675538008
0.7028715793638962
0.7521045742768074
0.7561520855194468
0.6595407969894826
0.7643516787502256
0.802804246133417
0.7684578962360368
1.0037065643409415
0.8530333097206295
0.7537063774551895
0.5923481898585635
0.37486082235043083
0.3679244202992168
0.3525412476445036
0.2021407199773441
0.7713142787992675
-0.00816239675761727
-0.16116791962759852
-0.11628259092936531
-0.2413019346807793
-0.1956980146986723
-0.008003830270838645
-0.08265050518654397
0.24685693005551002
0.17438964127487147
0.2597659060274606
0.17322723329944212
0.5259591405019027
0.5082713283163524
0.43081817730155164
0.3604283202016825
0.36664902960270596
0.29274467550313515
0.3802482525759165
0.45880786032902704
-0.008721619497859356
0.4610094872914157
0.5805562025163611
0.3065908989013708
0.5412983810810333


C:\Users\User\AppData\Local\Temp\ipykernel_1544\1482625837.py:20: RuntimeWarning: invalid value encountered in true_divide
  std_rets= (rets-mean_vec)/sd_vec


0.7377247614991673
0.07961106210368485
-0.25603930859473456
0.04829525340394871
-0.27114220079409695
-0.2755405415954576
0.160703482204392
-0.28622868869547624
-0.20046700829583394
-0.22999822799764977
0.13729554030286503
-0.25885140329662304
-0.2254830686955822
-0.14367184199559055
-0.27141610289779944
0.1326213090031274
0.09012373290093303
0.09948561719934876
0.020139509400170705
-0.00591101609939361
0.12681518100009168
-0.11026224809882916
-0.1210155152993779
-0.10942463459872265
-0.0837553847990132
-0.1254362255997939
-0.3005242904999149
-0.29423796389833257
0.1356859305002763
-0.04052856299935415
-0.033012410998904756
-0.04868448269959913
0.3564092666995304
-0.06076898370019503
-0.07774445158711657
-0.0728395658991337
-0.04378345935395318
-0.001621043098318144
0.00219975680577272
0.018689965802035975
0.07944455354454477
0.06207088550749655
0.34045607298160113
0.42615289107288307
0.09555952140175833
0.0738501702024228
0.6646736965820738
0.5913574862078189
0.011298088800949258
0.072